## AutoGen Studio Agent Workflow API Example

This notebook focuses on demonstrating capabilities of the autogen studio workflow python api.  

- Declarative Specification of an Agent Team
- Loading the specification and running the resulting agent

 

In [1]:
from autogenstudio.teammanager import TeamManager

wm = TeamManager()
result = await wm.run(task="What is the weather in New York?", team_config="team.json")
print(result)

task_result=TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='writing_agent', models_usage=RequestUsage(prompt_tokens=65, completion_tokens=15), content=[FunctionCall(id='call_jcgtAVlBvTFzVpPxKX88Xsa4', arguments='{"city":"New York"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='writing_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_jcgtAVlBvTFzVpPxKX88Xsa4')], type='ToolCallExecutionEvent'), TextMessage(source='writing_agent', models_usage=None, content='The weather in New York is 73 degrees and Sunny.', type='TextMessage'), TextMessage(source='writing_agent', models_usage=RequestUsage(prompt_tokens=103, completion_tokens=14), content='The current weather in New York is 73 degrees and sunny.', type='TextMessage')], stop_reason='Maximum number of messag

In [2]:
result_stream =  wm.run_stream(task="What is the weather in New York?", team_config="team.json")
async for response in result_stream:
    print(response)

source='user' models_usage=None content='What is the weather in New York?' type='TextMessage'
source='writing_agent' models_usage=RequestUsage(prompt_tokens=65, completion_tokens=15) content=[FunctionCall(id='call_EwdwWogp5jDKdB7t9WGCNjZW', arguments='{"city":"New York"}', name='get_weather')] type='ToolCallRequestEvent'
source='writing_agent' models_usage=None content=[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_EwdwWogp5jDKdB7t9WGCNjZW')] type='ToolCallExecutionEvent'
source='writing_agent' models_usage=None content='The weather in New York is 73 degrees and Sunny.' type='TextMessage'
source='writing_agent' models_usage=RequestUsage(prompt_tokens=103, completion_tokens=14) content='The weather in New York is currently 73 degrees and sunny.' type='TextMessage'
task_result=TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='w

## AutoGen Studio Database API

Api for creating objects and serializing to a database.

In [3]:
from autogenstudio.database import DatabaseManager
import os
# delete database
# if os.path.exists("test.db"):
#     os.remove("test.db")

os.makedirs("test", exist_ok=True)
# create a database
dbmanager = DatabaseManager(engine_uri="sqlite:///test.db", base_dir="test")
dbmanager.initialize_database()

Response(message='Database is ready', status=True, data=None)

In [7]:

from sqlmodel import Session, text, select
from autogenstudio.datamodel.types import  ModelTypes,  TeamTypes,  AgentTypes,   ToolConfig, ToolTypes, OpenAIModelConfig, RoundRobinTeamConfig, MaxMessageTerminationConfig, AssistantAgentConfig, TerminationTypes

from autogenstudio.datamodel.db import Model, Team, Agent, Tool,LinkTypes

user_id = "guestuser@gmail.com" 

gpt4_model = Model(user_id=user_id, config= OpenAIModelConfig(model="gpt-4o-2024-08-06", model_type=ModelTypes.OPENAI).model_dump() )

weather_tool = Tool(user_id=user_id, config=ToolConfig(name="get_weather", description="Get the weather for a city", content="async def get_weather(city: str) -> str:\n    return f\"The weather in {city} is 73 degrees and Sunny.\"",tool_type=ToolTypes.PYTHON_FUNCTION).model_dump() )

adding_tool = Tool(user_id=user_id, config=ToolConfig(name="add", description="Add two numbers", content="async def add(a: int, b: int) -> int:\n    return a + b", tool_type=ToolTypes.PYTHON_FUNCTION).model_dump() )

writing_agent = Agent(user_id=user_id,
                      config=AssistantAgentConfig(
                          name="writing_agent",
                          tools=[weather_tool.config],
                          agent_type=AgentTypes.ASSISTANT,
                          model_client=gpt4_model.config
                          ).model_dump()
                    )

team = Team(user_id=user_id, config=RoundRobinTeamConfig(
    name="weather_team",
    participants=[writing_agent.config],
    termination_condition=MaxMessageTerminationConfig(termination_type=TerminationTypes.MAX_MESSAGES, max_messages=5).model_dump(),
    team_type=TeamTypes.ROUND_ROBIN
    ).model_dump()
)

with Session(dbmanager.engine) as session:
    session.add(gpt4_model)
    session.add(weather_tool)
    session.add(adding_tool)
    session.add(writing_agent)
    session.add(team)
    session.commit()

    dbmanager.link(LinkTypes.AGENT_MODEL, writing_agent.id, gpt4_model.id)
    dbmanager.link(LinkTypes.AGENT_TOOL, writing_agent.id, weather_tool.id)
    dbmanager.link(LinkTypes.AGENT_TOOL, writing_agent.id, adding_tool.id)
    dbmanager.link(LinkTypes.TEAM_AGENT, team.id, writing_agent.id)


In [7]:
all_teams = dbmanager.get(Team)
print(len(all_teams.data), "teams in database")

2 teams in database


## Configuration Manager

Helper class to mostly import teams/agents/models/tools etc into a database.

In [8]:
from autogenstudio.database import ConfigurationManager

config_manager = ConfigurationManager(dbmanager)


In [9]:
result = await config_manager.import_component("team.json", user_id="user_id", check_exists=True)
print(result)

message='Team Created Successfully' status=True data={'id': 4, 'updated_at': datetime.datetime(2024, 12, 15, 15, 52, 21, 674916), 'version': '0.0.1', 'created_at': datetime.datetime(2024, 12, 15, 15, 52, 21, 674910), 'user_id': 'user_id', 'config': {'version': '1.0.0', 'component_type': 'team', 'name': 'weather_team', 'participants': [{'version': '1.0.0', 'component_type': 'agent', 'name': 'writing_agent', 'agent_type': 'AssistantAgent', 'description': None, 'model_client': {'version': '1.0.0', 'component_type': 'model', 'model': 'gpt-4o-2024-08-06', 'model_type': 'OpenAIChatCompletionClient', 'api_key': None, 'base_url': None}, 'tools': [{'version': '1.0.0', 'component_type': 'tool', 'name': 'get_weather', 'description': 'Get the weather for a city', 'content': 'async def get_weather(city: str) -> str:\n    return f"The weather in {city} is 73 degrees and Sunny."', 'tool_type': 'PythonFunction'}], 'system_message': None}], 'team_type': 'RoundRobinGroupChat', 'termination_condition': {

In [10]:
result = await config_manager.import_directory(".", user_id="user_id", check_exists=False)
print(result)

message='Directory import complete' status=True data=[{'component': 'team', 'status': True, 'message': 'Team Created Successfully', 'id': 5}]


In [11]:
all_teams = dbmanager.get(Team)
print(len(all_teams.data), "teams in database")

5 teams in database


## Sample AgentChat Example (Python)

In [12]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

planner_agent = AssistantAgent(
    "planner_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request. Respond with a single sentence",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided. Respond with a single sentence",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.Respond with a single sentence",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed tfinal travel plan. You must ensure th b at the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.Respond with a single sentence",
)

termination = TextMentionTermination("TERMINATE")
group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent], termination_condition=termination
)

In [13]:

result = group_chat.run_stream(task="Plan a 3 day trip to Nepal.")
async for response in result:
    print(response)

source='user' models_usage=None content='Plan a 3 day trip to Nepal.' type='TextMessage'
source='planner_agent' models_usage=RequestUsage(prompt_tokens=45, completion_tokens=53) content='I recommend starting your trip in Kathmandu, where you can explore the historic Durbar Square and Pashupatinath Temple, then take a scenic flight over the Everest range, and finish your journey with a stunning hike in the Annapurna region.' type='TextMessage'
source='local_agent' models_usage=RequestUsage(prompt_tokens=115, completion_tokens=53) content='I recommend starting your trip in Kathmandu, where you can explore the historic Durbar Square and Pashupatinath Temple, then take a scenic flight over the Everest range, and finish your journey with a stunning hike in the Annapurna region.' type='TextMessage'
source='language_agent' models_usage=RequestUsage(prompt_tokens=199, completion_tokens=42) content="For your trip to Nepal, it's crucial to learn some phrases in Nepali since English is not widely